In [1]:
from IPython.display import display, Math
import sympy as sy
import numpy as np

x1, x2, x3, x4, x5, x6, x7, x8, x9, a, b = sy.symbols('x1, x2, x3, x4, x5, x6, x7, x8, x9, a, b')

# Abbruchkriterium a): Abbruch nach einer bestimmten Anzahl Iterationen
def is_finished_max_iterations(f, x, n_max):
    return x.shape[0] - 1 >= n_max

# Abbruchkriterium b): Abbruch, wenn ‖x(n+1) - x(n)‖₂ ≤ ‖x(n+1)‖₂ * 𝛜
def is_finished_relative_error(f, x, eps):
    if x.shape[0] < 2:
        return False
    return np.linalg.norm(x[-1] - x[-2], 2) <= np.linalg.norm(x[-1], 2) * 1.0 * eps

# Abbruchkriterium c): Abbruch, wenn ‖x(n+1) - x(n)‖₂ ≤ 𝛜
def is_finished_absolute_error(f, x, eps):
    if x.shape[0] < 2:
        return False
    return np.linalg.norm(x[-1] - x[-2], 2) <= 1.0 * eps

# Abbruchkriterium d): Abbruch, wenn ‖f(x(n+1))‖₂ ≤ 𝛜
def is_finished_max_residual(f, x, eps):
    if x.shape[0] < 1:
        return False
    return np.linalg.norm(f(x[-1]), np.inf) <= 1.0 * eps

"""
=======================================================================================================================
INPUT
=======================================================================================================================
"""

# ACHTUNG: Für sinus/cosinus/Exponentialfunktion immer sy.sin/sy.cos/sy.exp/sy.ln/sy.abs verwenden!

f = sy.Matrix([
    1 - x1 ** 2 - x2 ** 2,
    (((x1 - 2) ** 2) / 2) + (((x2 - 1) ** 2) / 4) - 1
])

x = sy.Matrix([x1, x2])

 # Startwert
x0 = np.array([2, -1])

# Wähle das Abbruchkriterium (bei passender Zeile Kommentar entfernen):
def is_finished(f, x):
    # return is_finished_max_iterations(f, x, 1)      # Criterion a)
    # return is_finished_relative_error(f, x, 1e-5)  # Criterion b)
    # return is_finished_absolute_error(f, x, 1e-4)  # Criterion c)
    return is_finished_max_residual(f, x, 1e-8)    # Criterion d)

"""
=======================================================================================================================
"""

display(Math(r'\text{Bilde die Jacobi-Matrix } Df(x) \text{ für } f(x) = ' + sy.latex(f) + r' \text{ mit } x = ' + sy.latex(x)))

# Bilde die allgemeine Jacobi-Matrix
Df = f.jacobian(x)

display(Math(r'\text{Ganze Jacobi-Matrix: } Df = ' + sy.latex(Df)))

# Sympy-Funktionen kompatibel mit Numpy machen
f_lambda = sy.lambdify([x], f, "numpy")
Df_lambda = sy.lambdify([x], Df, "numpy")

# Newton-Iterationen
x_approx = np.empty(shape=(0, len(x)), dtype=np.float64) # Array mit Lösungsvektoren x0 bis xn
x_approx = np.append(x_approx, [x0], axis=0) # Start-Vektor in Array einfügen
display(Math(r'x({0}) = ' + sy.latex(sy.Matrix(x0))))

while not is_finished(f_lambda, x_approx):
    i = x_approx.shape[0] - 1
    print()
    print(f'ITERATION {i + 1}')
    print('-------------------------------------')

    x_n = x_approx[-1] # x(n) (letzter berechneter Wert)

    display(Math(r'\delta_{' + str(i) + r'} \text{ ist die Lösung des LGS } Df(x_{' + str(i) + r'}) * \delta_{' + str(i) + r'} = -1 * f(x_{' + str(i) + r'})'))
    display(Math(sy.latex(sy.Matrix(Df_lambda(x_n))) + '\\cdot\\delta_{' + str(i) + '}=-1\\cdot' + sy.latex(sy.Matrix(f_lambda(x_n)))))

    Q, R = np.linalg.qr(Df_lambda(x_n))
    delta = np.linalg.solve(R, -Q.T @ f_lambda(x_n)).flatten()  # 𝛅(n) aus Df(x(n)) * 𝛅(n) = -1 * f(x(n))
    display(Math(r'\delta_{' + str(i) + r'} = ' + sy.latex(sy.Matrix(delta))))

    x_next = x_n + delta.reshape(x0.shape[0], )  # x(n+1) = x(n) + 𝛅(n)

    print()
    display(Math(r'x_{' + str(i + 1) + r'} = x_{' + str(i) + r'} + \delta_{' + str(i) + r'}'))
    display(Math(r'x_{' + str(i + 1) + r'} = ' + sy.latex(sy.Matrix(x_next))))
    display(Math(r'\|f(x_{' + str(i + 1) + r'})\|_2 = ' + str(np.linalg.norm(f_lambda(x_next), 2))))
    display(Math(r'\|x_{' + str(i + 1) + r'} - x_{' + str(i) + r'}\|_2 = ' + str(np.linalg.norm(x_next - x_n, 2))))

    x_approx = np.append(x_approx, [x_next], axis=0)

print()
print('-------------------------------------')
print()

display(Math(r'\text{Die approximierten Werte sind: }' + sy.latex(sy.Matrix(x_approx))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


ITERATION 1
-------------------------------------


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


-------------------------------------



<IPython.core.display.Math object>